<a href="https://colab.research.google.com/github/pranavrajaram/datascience/blob/main/Car_Type.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
###DO NOT EDIT THIS CODE
################################################################################################################################
import cv2
import requests
import urllib.request
import urllib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt

# GPUs are 3x faster than CPU. Better to use if it is available 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define Loss Function
loss_function = nn.CrossEntropyLoss()

# This function returns the number of parameters in the model
def num_params(model):
  return sum([p.numel() for p in model.parameters()])

# Define a Training Function. This function will: compute the forward pass, backpropagate,
# update the weights, and repeat the steps for a given number of epochs. At each epoch, 
# it will output the training loss and test loss at every step
def train(epochs, model, trainloader, testloader, optimizer, loss_function):
  for epoch in range(epochs):
    loss_epoch = np.array([])
    train_correct, train_total = 0, 0
    test_correct, test_total = 0, 0

    for data, labels in trainloader:
      # convert into GPU objects if needed
      input_data = data.to(device)
      labels = labels.to(device)

      # forward pass
      predict = model(input_data)
      
      # backward pass
      loss = loss_function(predict, labels)
      optimizer.zero_grad()
      loss.backward()

      # update parameters (weights and biases)
      optimizer.step()

      # store progress
      loss_epoch = np.append(loss_epoch, loss.item())

    # evaluate test accuracy
    for data, labels in testloader:
      input_data = data.to(device)
      labels = labels.to(device)
      predict = model(input_data)
      for i, out in enumerate(predict):
        pred = torch.argmax(out)
        if pred == labels[i]:
          test_correct+=1
        test_total+=1

    test_accuracy = test_correct/test_total    
  
    print('epoch [{}/{}], training loss:{:.4f}, test accuracy:{:.4f}'.format(epoch+1, epochs, np.mean(loss_epoch), test_accuracy))
################################################################################################################################

In [8]:
# download and load data
batch_size = 512


# download and transform train dataset
train_loader = torch.utils.data.DataLoader(datasets.EMNIST('./emnist_data', split='mnist', download=True, train=True, transform=transforms.Compose([
                                                transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                ])), batch_size=batch_size, shuffle=True)

# download and transform test dataset
test_loader = torch.utils.data.DataLoader(datasets.EMNIST('./emnist_data', split='mnist', download=True, train=False, transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), batch_size=batch_size, shuffle=True)

In [9]:
########################
####                #### 
#### EDIT THIS CELL ####
####                ####
########################


learning_rate = 42e-4
weight_decay = 10e-5
n_epochs = 9

# neural network
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    ### Define Layers
    self.c1 = nn.Conv2d(1, 3, 5)
    self.L1 = nn.Linear(1728, 723)
    self.L2 = nn.Linear(723, 98)
    self.L3 = nn.Linear(98, 10)

  def forward(self, x):
    x = self.c1(x)
    x = F.relu(x)
    x = torch.flatten(x,1)
    x = self.L1(x)
    x = F.relu(x)
    x = self.L2(x)
    x = F.relu(x)
    x = self.L3(x)
    x = torch.sigmoid(x)
    return x
  
# Every time you edit the neural network, you'll have to update this cell
# Create model object
model = NeuralNetwork().to(device)

# Loads Adam optimizer, which implements a version of gradient descent
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [10]:
train(n_epochs, model, train_loader, test_loader, optimizer, loss_function)

epoch [1/9], training loss:1.5531, test accuracy:0.9458
epoch [2/9], training loss:1.4974, test accuracy:0.9627
epoch [3/9], training loss:1.4920, test accuracy:0.9672
epoch [4/9], training loss:1.4867, test accuracy:0.9595
epoch [5/9], training loss:1.4856, test accuracy:0.9644
epoch [6/9], training loss:1.4828, test accuracy:0.9728
epoch [7/9], training loss:1.4803, test accuracy:0.9716
epoch [8/9], training loss:1.4798, test accuracy:0.9757
epoch [9/9], training loss:1.4780, test accuracy:0.9715
